<a href="https://colab.research.google.com/github/romoreira/distributed_learning/blob/main/TRAILS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install datatime

ERROR: Could not find a version that satisfies the requirement datatime (from versions: none)
ERROR: No matching distribution found for datatime


In [2]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import image_data_format
from datetime import datetime

import logging
import threading
import time

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import sys
import statistics

from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

from build_model import Model
import csv

# client config
NUMOFCLIENTS = 2 # number of client(as particles)
SELECT_CLIENTS = 0.5 # c
EPOCHS = 1 # number of total iteration
CLIENT_EPOCHS = 5 # number of each client's iteration
BATCH_SIZE = 10 # Size of batches to train on
DROP_RATE = 0

# model config 
LOSS = 'categorical_crossentropy' # Loss function
NUMOFCLASSES = 10 # Number of classes
lr = 0.0025
# OPTIMIZER = SGD(lr=0.015, decay=0.01, nesterov=False)
OPTIMIZER = SGD(lr=lr, momentum=0.9, decay=lr/(EPOCHS*CLIENT_EPOCHS), nesterov=False) # lr = 0.015, 67 ~ 69%

def load_dataset():
    # Code for experimenting with CIFAR-10 datasets.
    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
    
    # Code for experimenting with MNIST datasets.
    # (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    # X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    # X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
    
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    Y_train = to_categorical(Y_train)
    Y_test = to_categorical(Y_test)

    return (X_train, Y_train), (X_test, Y_test)

def init_model(train_data_shape):
    print("Data Shape: "+str(train_data_shape))
    model = Model(loss=LOSS, optimizer=OPTIMIZER, classes=NUMOFCLASSES)
    fl_model = model.fl_paper_model(train_shape=train_data_shape)

    return fl_model



#lists of federated_clients
federated_clients_as_threads = []
fed_clients_current_round = []

#Global Training Model Time
global_run_time = 1
global_avg = 0
(x_train, y_train), (x_test, y_test) = load_dataset()
server_model = init_model(train_data_shape=x_train.shape[1:])
server_weights = []



def utf8len(s):
    return len(s.encode('utf-8'))

def write_csv(method_name, list):
    file_name = '{name}_CIFAR10_randomDrop_{drop}%_output_C_{c}_LR_{lr}_CLI_{cli}_CLI_EPOCHS_{cli_epoch}_TOTAL_EPOCHS_{epochs}_BATCH_{batch}.csv'
    file_name = file_name.format(folder="origin_drop",drop=DROP_RATE, name=method_name, c=SELECT_CLIENTS, lr=lr, cli=NUMOFCLIENTS, cli_epoch=CLIENT_EPOCHS, epochs=EPOCHS, batch=BATCH_SIZE)

    save_path = "/content/gdrive/MyDrive/Colab Notebooks"
    completeName = os.path.join(save_path, file_name)

    f = open(completeName, 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)
    
    

    for l in list:
        wr.writerow(l)
    
    f.close()






def client_data_config(x_train, y_train):
    client_data = [() for _ in range(NUMOFCLIENTS)] # () for _ in range(NUMOFCLIENTS)
    num_of_each_dataset = int(x_train.shape[0] / NUMOFCLIENTS)

    print("Size of x_train: "+str(len(x_train)))

    print("Num_of_each_dataset: "+str(num_of_each_dataset))
    
    for i in range(NUMOFCLIENTS):
        split_data_index = []
        while len(split_data_index) < num_of_each_dataset:
            item = random.choice(range(x_train.shape[0]))
            if item not in split_data_index:
                split_data_index.append(item)
        
        new_x_train = np.asarray([x_train[k] for k in split_data_index])
        new_y_train = np.asarray([y_train[k] for k in split_data_index])
    
        client_data[i] = (new_x_train, new_y_train)

    return client_data


def fedAVG(server_weight):
    #print("Server_weight[0]): "+str(server_weight[0]))
    avg_weight = np.array(server_weight[0])
    print("len(Server_weight[0]): "+str(len(server_weight)))

    if len(server_weight) > 1:
        for i in range(1, len(server_weight)):
            #print("Each i of server_weight: "+str(server_weight[i]))
            avg_weight += server_weight[i]
    
    avg_weight = avg_weight / len(server_weight)

    return avg_weight


def client_update(index, client, avg_weight, x_test, y_test):
    print("Fed_Client Thread {}/{} fitting\n".format(index + 1, int(NUMOFCLIENTS * SELECT_CLIENTS)))

    if fed_clients_current_round[index] != 0:
      print("Client: "+str(index)+ " is Sleeping for: "+str(fed_clients_current_round[index]))
      time.sleep(fed_clients_current_round[index])
    

    federated_clients_as_threads[index][2] = federated_clients_as_threads[index][2] + 1#Incrementa o Round que o FED_Client já está treinando

    

    time_register(index)
    client.fit(client_data[index][0], client_data[index][1],
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          verbose=0,
          validation_split=0.2,
    )
    time_register(index)

    print("\n\n\nEnd of training Thread: "+str(index))
    print("\n\n\n Validating the client training: "+str(index))
    scores = fed_client_evaluation(client, x_test, y_test)
    print("\n\nThe accuracy of Client "+str(index)+" is: "+str(scores)+" \n\n\n\n")
    send_model_to_server(scores, index, client)

def fed_client_evaluation(model, x_test, y_test):
    return model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)


def send_model_to_server(metric_scores, client_index, fed_model_trained):
    print("\n\n\nEnviando metricas para o server. Cliente: "+str(client_index))
    global server_weights
    federated_clients_as_threads[client_index].append(metric_scores)
    federated_clients_as_threads[client_index].append(fed_model_trained)
    server_weights.append(copy.deepcopy(fed_model_trained.get_weights()))

    

def waiter():
    all_fed_finished = False
    run_time_training_acc = []
    global global_run_time
    global fed_clients_current_round
    global global_avg
    global server_model
    global server_weights
    while(True):
        if global_run_time == 1:
          print("\n Current status of Global Training: "+str(federated_clients_as_threads))
          time.sleep(10)
          for i in range(len(federated_clients_as_threads)):
              if federated_clients_as_threads[i][0].is_alive() == True:
                  all_fed_finished = False
                  break
              else:
                all_fed_finished = True
          if all_fed_finished:
              print("Todos os federados do Round terminaram a rodada de treinamento. Calcular os tempos e por pra rodar denovo")
              avg = []
              for i in range (len(federated_clients_as_threads)):
                  avg.append(federated_clients_as_threads[i][4][1])
              print("AVG: "+str(statistics.mean(avg)))
              global_avg = statistics.mean(avg)
              print("STD: "+str(statistics.stdev(avg)))
              
              avg_weight = fedAVG(server_weights)
              server_model.set_weights(avg_weight)
              server_weights = []
              
              global_run_time = global_run_time + 1
              for i in range(len(federated_clients_as_threads)):
                print("Status do fed na interação 1: "+str(federated_clients_as_threads[i][0]))
                fed_clients_current_round.append(0)#Reseta para que apos a inicializacao todos os feds tenham oportundiade de treinar uma segunda vez
                #federated_clients_as_threads[i][4] = []#Reseta a lista de metricas na segunda interação
                federated_clients_as_threads[i][0].start()
        else:
          for i in range(len(federated_clients_as_threads)):
            if not(federated_clients_as_threads[i][0].is_alive()):
              federated_clients_as_threads[i][0].stat()
          
          while(True):
            print("\n Current status of Global Training: "+str(federated_clients_as_threads))
            time.sleep(10)
            for i in range(len(federated_clients_as_threads)):
                if federated_clients_as_threads[i][0].is_alive() == True:
                    all_fed_finished = False
                    break
                else:
                  all_fed_finished = True
            if all_fed_finished:
              print("Todos os federados do Round terminaram a rodada de treinamento ou estao dormindo penalizados. Calcular os tempos e por pra rodar denovo")
              acc_client = []
              run_time_average = []
              for i in range (len(federated_clients_as_threads)):
                  acc_client.append(federated_clients_as_threads[i][4][1])
                  run_time_average.append(federated_clients_as_threads[i][3][1]-federated_clients_as_threads[i][3][1])
                  print("Tempo de treinamento medio global: "+str(statistics.mean(run_time_average)))
              
              for i in range (len(federated_clients_as_threads)):
                  if federated_clients_as_threads[i][4][1] >= global_avg:
                    federated_clients_as_threads[i][2] = EPOCHS * 2
                    fed_clients_current_round[i] = 0
                  else:
                    federated_clients_as_threads[i][2] = EPOCHS+1
                    fed_clients_current_round[i] = statistics.mean(run_time_average)

              
              print("AVG: "+str(statistics.mean(acc_client)))
              global_avg = statistics.mean(acc_client)
              print("STD: "+str(statistics.stdev(acc_client)))
              
              avg_weight = fedAVG(server_weights)
              server_model.set_weights(avg_weight)
              server_weights = []
              
              global_run_time = global_run_time + 1
              

          

def fed_client_selection(policy):

    if policy == "TRAILS":
        print("Decidir qual cliente escolher")
        #https://github.com/dnanhkhoa/simple-bloom-filter

def time_register(index_fed_client):
      now = datetime.now()
      #current_time = now.strftime("%H:%M:%S")
      federated_clients_as_threads[index_fed_client][3].append(now)   

if __name__ == "__main__":
    
    
    federated_clients_as_threads = []

    
    server_model.summary()

    client_data = client_data_config(x_train, y_train)
    print("Client_data: "+str(len(client_data)))
    fl_models = []
    for i in range(NUMOFCLIENTS):
        fl_models.append(init_model(train_data_shape=client_data[i][0].shape[1:]))


    avg_weight = np.zeros_like(server_model.get_weights())
    print("AVG_Weight: "+str(avg_weight))
    server_evaluate_acc = []

    print("NUMOFCLIENTS: "+str(NUMOFCLIENTS))
    print("Select_clients: "+str(SELECT_CLIENTS))


    waiter = threading.Thread(target=waiter, args=(), daemon=True)
    waiter.start()
    
      
    for index, client in enumerate(fl_models):
   
      #print("Index: "+str(index))
      #print("CLIENT: "+str(client))
      fed = []
      a = threading.Thread(target=client_update, args=(index, client, avg_weight, x_test, y_test,))
      fed.append(a)
      fed.append(index)
      fed.append(0)#Round que o cliente já está treinando

      federated_clients_as_threads.append(fed)

    for i in range(len(federated_clients_as_threads)):
        fed_clients_current_round.append(0)
        federated_clients_as_threads[i].append([])
        federated_clients_as_threads[i][0].start()

    waiter.join()
    print(federated_clients_as_threads)
    for i in range(len(federated_clients_as_threads)):
        federated_clients_as_threads[i][0].join()
    

#    write_csv("FedAvg", server_evaluate_acc)
      
    
  

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Data Shape: (32, 32, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        51264     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        102464    
                                

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Fed_Client Thread 2/1 fitting


 Current status of Global Training: [[<Thread(Thread-12, started 139883280377600)>, 0, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 685181)]], [<Thread(Thread-13, started 139883271984896)>, 1, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 735881)]]]

 Current status of Global Training: [[<Thread(Thread-12, started 139883280377600)>, 0, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 685181)]], [<Thread(Thread-13, started 139883271984896)>, 1, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 735881)]]]

 Current status of Global Training: [[<Thread(Thread-12, started 139883280377600)>, 0, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 685181)]], [<Thread(Thread-13, started 139883271984896)>, 1, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 735881)]]]

 Current status of Global Training: [[<Thread(Thread-12, started 139883280377600)>, 0, 2, [datetime.datetime(2021, 12, 3, 20, 1, 35, 685181)]], [<Thread(Thread-13, started 139883271984896)>, 1, 2, [datetime.datet

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:137: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:143: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-24a0db975e42>", line 226, in waiter
    federated_clien